In [1]:
import h5py, sys, os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import matplotlib.colors as colors
from datetime import datetime
print(os.getcwd())

%matplotlib ipympl

from jfunctions import *

/mhome/damtp/q/cwp29/diablo3/proc/jupyter


In [2]:
#Load simulation data
b, phi, W, Scum, F_b, F_phi, M, times = load_data('th1_xz', 'th2_xz', 'td_scatter', 'td_flux', 'td_vel_1', 'td_vel_2', 'pvd')
NSAMP = len(times)

# Load simulation metadata
md = get_metadata()

# Get dir locations from param file
base_dir, run_dir, save_dir, version = read_params('./params.dat')
print(base_dir)

bbins, phibins = load_bins(save_dir)

times, t0_idx = calibrate_time(save_dir, times)


# Load grids
gxf, gyf, gzf, dzf = get_grid(md)
gx, gy, gz, dz = get_grid(md, fractional_grid=False)

Keys: <KeysViewHDF5 ['B_xy', 'B_xz', 'B_yz', 'Ent_phic_flux_int', 'Ent_phic_flux_rec', 'Ent_phiv_flux_int', 'Ent_phiv_flux_rec', 'N2_xy', 'N2_xz', 'N2_yz', 'Re_b_xy', 'Re_b_xz', 'Re_b_yz', 'Ri_xy', 'Ri_xz', 'Ri_yz', 'b_phic_F1', 'b_phic_F3', 'b_phic_M', 'b_phic_S', 'b_phic_W', 'b_phic_mF1', 'b_phic_mF3', 'b_phic_sF3', 'b_phiv_F1', 'b_phiv_F2', 'b_phiv_M', 'b_phiv_S', 'b_phiv_W', 'b_phiv_mF1', 'b_phiv_mF2', 'chi1_xy', 'chi1_xz', 'chi1_yz', 'chi2_xy', 'chi2_xz', 'chi2_yz', 'chi3_xy', 'chi3_xz', 'chi3_yz', 'diff_th1_xy', 'diff_th1_xz', 'diff_th1_yz', 'diff_th2_xy', 'diff_th2_xz', 'diff_th2_yz', 'diff_th3_xy', 'diff_th3_xz', 'diff_th3_yz', 'epsilon_xy', 'epsilon_xz', 'epsilon_yz', 'kappa_t1_xy', 'kappa_t1_xz', 'kappa_t1_yz', 'kappa_t2_xy', 'kappa_t2_xz', 'kappa_t2_yz', 'kappa_t3_xy', 'kappa_t3_xz', 'kappa_t3_yz', 'nu_t_xy', 'nu_t_xz', 'nu_t_yz', 'omegaX_xy', 'omegaX_xz', 'omegaX_yz', 'omegaY_xy', 'omegaY_xz', 'omegaY_yz', 'omegaZ_xy', 'omegaZ_xz', 'omegaZ_yz', 'th1_xy', 'th1_xz', 'th1_yz',

KeyError: "Unable to open object (object 'td_scatter' doesn't exist)"

In [ ]:
print(md)

In [ ]:
#Create grids
X, Y = np.meshgrid(gx, gz)
Xf, Yf = np.meshgrid(gxf, gzf)

db = bbins[1] - bbins[0]
dphi = phibins[1] - phibins[0]

sx, sy = np.meshgrid(np.append(bbins-db/2, bbins[-1]+db/2),
        np.append(phibins - dphi/2, phibins[-1] + dphi/2))
sxf, syf = np.meshgrid(bbins, phibins)

print("Total time steps: %s"%NSAMP)
print("Dimensional times: ",times)


In [ ]:
# Set-up
for i in range(1, NSAMP):
    Scum[i] += Scum[i-1]

S = np.gradient(Scum, md['SAVE_STATS_DT'], axis=0)
div_F = np.gradient(F_b, bbins, axis=2) + np.gradient(F_phi, phibins, axis=1)

W = np.where(W == 0, np.NaN, W)
S = np.where(S == 0, np.NaN, S)
M = np.where(M == 0, np.NaN, M)
F_b = np.where(F_b == 0, np.NaN, F_b)
F_phi = np.where(F_phi == 0, np.NaN, F_phi)
M = np.where(M == 0, np.NaN, M)
Scum = np.where(Scum == 0, np.NaN, Scum)
div_F = np.where(div_F == 0, np.NaN, div_F)

# Non-dimensionalising
F0 = compute_F0(save_dir, md, tstart_ind = 2*4, verbose=False, zbot=0.7, ztop=0.95, plot=False)
N = np.sqrt(md['N2'])
T = np.power(N, -1)
L = np.power(F0, 1/4) * np.power(N, -3/4)

# CURRENTLY OMITTED!

In [ ]:
# Thresholding
tracer_thresh = 5e-4
tracer_thresh_low = 1e-3
plot_plume = np.where(
        np.logical_or(
            np.logical_and(phi > tracer_thresh_low, Yf < md['H']-L),
            np.logical_and(phi > tracer_thresh, Yf >= md['H']-L)),
        phi, np.NaN)
plot_env = np.where(np.logical_and(np.isnan(plot_plume), Yf >= md['H']-L), b, np.NaN)

b = np.where(b < 1e-4, 0, b)

In [ ]:
# Create colourmaps
S_bounds = np.linspace(-1e-6,  1e-6, 9)
S_norm = colors.BoundaryNorm(boundaries=S_bounds, ncolors=256)

colors_red = plt.cm.coolwarm(np.linspace(0.53, 1, 32))
colors_blue = plt.cm.coolwarm(np.linspace(0, 0.47, 32))
all_colors = np.vstack((colors_blue, colors_red))
custom_cmap = colors.LinearSegmentedColormap.from_list("cmap", all_colors)

In [ ]:
# Set plot limits and contours
bmax_plot = 1e-1#bbins[-1]
phimax_plot = 5e-2#phibins[-1]

contours_b = np.linspace(1e-4, bmax_plot, 10)[1:]
contours_phi = np.linspace(phibins[0] - dphi/2, phimax_plot, 10)
mid_tracer_thresh = contours_phi[2]

In [ ]:
# Compute zmax
tracer_data_vert = np.where(phi[:, :, int(md['Nx']/2)] >= mid_tracer_thresh,
        phi[:, :, int(md['Nx']/2)], 0)
plume_vert = np.where(tracer_data_vert >= mid_tracer_thresh, 1, 0)
heights = []
for i in range(len(plume_vert)):
    stuff = np.where(plume_vert[i] == 1)[0]
    if len(stuff) == 0:
        heights.append(0)
    else:
        heights.append(gzf[np.max(stuff)+1])

In [ ]:
def decorate(fig, axs, step):
    fig.suptitle("t = {0:.0f} s".format(times[step]))

    for a in axs.ravel()[1:]:
        a.set_xlim(bbins[0]-db/2, bmax_plot)
        a.set_ylim(phibins[0]-dphi/2, phimax_plot)

        a.set_xlabel(r"$b$")
        a.set_ylabel(r"$\phi$")

    axs[0,0].set_xlim(0, 0.4)
    axs[0,0].set_aspect(1)
    axs[0,0].set_ylim(md['H']-L, md['H']+7*L)
    axs[0,0].set_xlabel(r"$x$")
    axs[0,0].set_ylabel(r"$z$")

    im_phi.set_clim(phibins[0]-dphi/2, phimax_plot)
    cont_phi.set_clim(phibins[0]-dphi/2, phimax_plot)
    im_W.set_clim(0, 1e-5)
    im_W_F.set_clim(0, 1e-5)

    axs[0,0].set_title(r"(a) tracer field $\phi$")
    axs[0,1].set_title(r"(b) volume distribution $W$")
    axs[0,2].set_title(r"(c) flux distribution $\mathbf{F}$")
    axs[1,0].set_title(r"(d) cumulative source distribution $\int S \, \mathrm{d}t$")
    axs[1,1].set_title(r"(e) source distribution $S$")
    axs[1,2].set_title(r"(f) cumulative mixed volume distribution $M$")

In [ ]:
def decorate_cb(fig, axs, step):
    cb_W = fig.colorbar(im_W, ax=axs[0,1], location='right', shrink=0.8)
    cb_W.set_label("$W$", rotation=0, labelpad=5)

    cb_env = fig.colorbar(im_b_edge, ax=axs[0,0], location='right', shrink=0.8)
    cb_env.set_label(r"$b$", rotation=0, labelpad=5)

    #cb_phi = fig.colorbar(cont_phi, ax=axs[0,0], location='right', shrink=0.8)
    #cb_phi.set_label(r"$\phi$", rotation=0, labelpad=5)

    cb_plume = fig.colorbar(im_phi, ax = axs[0,0], location='right', shrink=0.8, extend='max')
    cb_plume.set_label(r"$\phi$", rotation=0, labelpad=5)

    cb_Scum = fig.colorbar(im_Scum, ax=axs[1,0], location='right', shrink=0.8)
    cb_Scum.set_label("cumulative $S$", rotation=0, labelpad=5)

    cb_S = fig.colorbar(im_S, ax=axs[1,1], location='right', shrink=0.8)
    cb_S.set_label("$S$", rotation=0, labelpad=5)

    cb_M = fig.colorbar(im_M, ax=axs[1,2], location="right", shrink=0.8)
    cb_M.set_label("$M$", rotation=0, labelpad=5)

    cb_WF = fig.colorbar(im_W, ax = axs[0,2], location='right', shrink=0.8)
    cb_WF.set_label("$W$", rotation=0, labelpad=5)

In [ ]:
def plots(fig, axs, step):
    global im_b_edge, im_phi, im_W, im_Scum, im_S, im_M, im_W_F, cont_phi

    im_b_edge = axs[0,0].contour(Xf, Yf, plot_env[step], levels=contours_b, cmap='cool', alpha=0.8)
    im_phi = axs[0,0].pcolormesh(X, Y, plot_plume[step], cmap='viridis', alpha=0.5)
    cont_phi = axs[0,0].contour(Xf, Yf, plot_plume[step], cmap='viridis', levels=contours_phi, ls='--')

    im_W = axs[0,1].pcolormesh(sx, sy, W[step], cmap='plasma')
    im_Scum = axs[1,0].pcolormesh(sx, sy, Scum[step], cmap='coolwarm', norm=S_norm)
    im_S = axs[1,1].pcolormesh(sx, sy, S[step], cmap='coolwarm', norm=S_norm)

    M_lim = np.nanmax(M[step])
    im_M = axs[1,2].pcolormesh(sx, sy, M[step], cmap=custom_cmap,
            norm=colors.CenteredNorm(halfrange = .6*M_lim))

    im_W_F = axs[0,2].pcolormesh(sx, sy, W[step], cmap='plasma', alpha=0.5)

    fn = 2 # filter_num
    im_F = axs[0,2].quiver(sxf[::fn,::fn], syf[::fn, ::fn], F_b[step, ::fn, ::fn],
            F_phi[step, ::fn, ::fn], angles='xy', units='xy', pivot='mid',
            fc='k', ec='k', linewidth=0.1)

    decorate(fig, axs, step)

In [ ]:
# Plotting loop
fig, axs = plt.subplots(2, 3, figsize=(18, 8), constrained_layout=True)

plots(fig, axs, -1)
decorate_cb(fig, axs, -1)

def animate(step):
    for a in axs.ravel():
        a.clear()

    plots(fig, axs, step)
    decorate(fig, axs, step)

Writer = animation.writers['ffmpeg']
writer = Writer(fps=4, bitrate=-1)

anim = animation.FuncAnimation(fig, animate, interval=250, frames=list(range(t0_idx, NSAMP)), repeat=False)
now = datetime.now()
#anim.save(save_dir+'scatter_%s.mp4'%now.strftime("%d-%m-%Y"),writer=writer)
plt.show()